In [1]:
from bs4 import BeautifulSoup
import requests
from IPython.core.display import display, HTML
import re
import dateutil.parser
import pandas as pd
import time

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
%matplotlib 

Using matplotlib backend: MacOSX


In [2]:
#2016 does not have needed expanded statistics
df_all=pd.DataFrame()
for y in [x for x in range(2010,2021) if x!=2016]:
    time.sleep(5)
    stats_year = y
    for i in range(1,282,40):
        url = 'https://www.espn.com/golf/statistics/_/year/'+ str(stats_year) + "/count/{}".format(i)
#        print(url)
        page = requests.get(url)
        soup = BeautifulSoup(page.text, "html.parser")
        header = soup.find("tr",class_="colhead")
        columns = [col.get_text() for col in header.find_all("td")]
        golfers = soup.find_all("tr", re.compile('row player-1106-'))
        for golfer in golfers:
            stats = [stat.get_text() for stat in golfer.find_all("td")]
            df_temp=pd.DataFrame(stats).T
            df_temp.columns=columns
            df_temp["YEAR"]= int(stats_year)
            df_all=pd.concat([df_all,df_temp],ignore_index=True) 
df_all

,RK,PLAYER,AGE,EVENTS,ROUNDS,CUTS MADE,TOP 10,WINS,CUP POINTS,EARNINGS,YEAR
0,1,Matt Kuchar,42,26,95,23,11,1,"2,728","$4,910,476.50",2010
1,2,Jim Furyk,50,21,74,18,7,3,"2,980","$4,809,622.50",2010
2,3,Ernie Els,51,20,70,17,7,2,"1,438","$4,558,861.00",2010
3,4,Dustin Johnson,36,23,82,20,7,2,"2,493","$4,473,122.00",2010
4,5,Steve Stricker,54,19,72,19,9,2,"2,028","$4,190,235.50",2010
...,...,...,...,...,...,...,...,...,...,...,...
3195,,Zander Winston,--,0,0,0,0,0,0,--,2020
3196,,Connor Howe,--,0,0,0,0,0,0,--,2020
3197,,Donnie Trosper,--,0,4,0,0,0,0,--,2020
3198,,Jared C. Kobren,--,0,0,0,0,0,0,--,2020


In [3]:
df_all_exp1=pd.DataFrame()
for y in [x for x in range(2010,2021) if x!=2016]:
    time.sleep(5)
    stats_year = y
    for i in range(1,162,40):
        url = 'https://www.espn.com/golf/statistics/_/year/'+ str(stats_year) + '/type/expanded/count/{}'.format(i)
#        print(url)
        page = requests.get(url)
        soup = BeautifulSoup(page.text, "html.parser")
        header = soup.find("tr",class_="colhead")
        columns = [col.get_text() for col in header.find_all("td")]
        golfers = soup.find_all("tr", re.compile('row player-1106-'))
        for golfer in golfers:
            stats = [stat.get_text() for stat in golfer.find_all("td")]
            df_temp_exp1=pd.DataFrame(stats).T
            df_temp_exp1.columns=columns
            df_temp_exp1["YEAR"]= int(stats_year)
            df_all_exp1=pd.concat([df_all_exp1,df_temp_exp1],ignore_index=True) 
df_all_exp1

,RK,PLAYER,AGE,YDS/DRIVE,DRIVING ACC.,DRVE TOTAL,GREENS IN REG.,PUTT AVG.,SAVE PCT.,YEAR
0,1,Luke Donald,43,277.0,62.4,297,65.3,1.751,65.8,2010
1,2,Carl Pettersson,43,282.1,62.8,249,58.6,1.736,62.9,2010
2,3,Pat Perez,45,287.1,65.2,172,69.0,1.761,62.0,2010
3,4,Tim Clark,45,272.2,73.7,192,66.7,1.756,61.6,2010
4,5,Chad Collins,42,275.8,60.3,324,57.5,1.751,60.4,2010
...,...,...,...,...,...,...,...,...,...,...
1868,189,Michael Gellerman,--,293.5,62.5,136,68.3,1.783,33.3,2020
1869,190,Martin Laird,38,302.1,63.7,46,70.5,1.806,32.2,2020
1870,191,Sam Burns,24,312.0,54.8,8,67.0,1.755,32.1,2020
1871,192,Vincent Whaley,--,300.9,58.4,65,66.7,1.761,29.4,2020


In [4]:
df_all_exp2=pd.DataFrame()
for y in [x for x in range(2010,2021) if x!=2016]:
    time.sleep(5)
    stats_year = y
    for i in range(1,162,40):
        url = 'https://www.espn.com/golf/statistics/_/year/'+ str(stats_year) + '/type/expanded2/count/{}'.format(i)
#        print(url)
        page = requests.get(url)
        soup = BeautifulSoup(page.text, "html.parser")
        header = soup.find("tr",class_="colhead")
        columns = [col.get_text() for col in header.find_all("td")]
        golfers = soup.find_all("tr", re.compile('row player-1106-'))
        for golfer in golfers:
            stats = [stat.get_text() for stat in golfer.find_all("td")]
            df_temp_exp2=pd.DataFrame(stats).T
            df_temp_exp2.columns=columns
            df_temp_exp2["YEAR"]= int(stats_year)
            df_all_exp2=pd.concat([df_all_exp2,df_temp_exp2],ignore_index=True) 
df_all_exp2

,RK,PLAYER,AGE,EAGLES,BIRDIES,PARS,BOGEYS,BIRDIES/RD,HOLES/EAGLE,YEAR
0,1,Ted Brown,--,1,6,21,8,3.000,36,2010
1,,Jim Herman,43,2,9,41,14,2.250,36,2010
2,,Jorge Campillo,34,1,3,25,6,1.500,36,2010
3,,Trevor Fisher,--,1,5,22,8,2.500,36,2010
4,,Ronnie Black,62,1,6,19,10,3.000,36,2010
...,...,...,...,...,...,...,...,...,...,...
1967,189,David Hearn,41,1,179,605,122,3.510,918,2020
1968,190,Brandt Snedeker,40,1,201,617,122,3.792,954,2020
1969,191,Zach Johnson,45,1,230,717,125,3.770,1098,2020
1970,192,Zac Blair,30,1,240,744,177,3.636,1188,2020


In [31]:
df_main_plus = df_all.merge(df_all_exp1,how='left',on=("PLAYER","YEAR"),suffixes=(None,"_drop1"))
#df_main_plus 

In [32]:
df_final = df_main_plus.merge(df_all_exp2,how='left',on=("PLAYER","YEAR"),suffixes=(None,"_drop2"))
df_final

,RK,PLAYER,AGE,EVENTS,ROUNDS,CUTS MADE,TOP 10,WINS,CUP POINTS,EARNINGS,...,PUTT AVG.,SAVE PCT.,RK_drop2,AGE_drop2,EAGLES,BIRDIES,PARS,BOGEYS,BIRDIES/RD,HOLES/EAGLE
0,1,Matt Kuchar,42,26,95,23,11,1,"2,728","$4,910,476.50",...,1.739,56.2,121,42,9,377,1111,199,3.968,190
1,2,Jim Furyk,50,21,74,18,7,3,"2,980","$4,809,622.50",...,1.767,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Ernie Els,51,20,70,17,7,2,"1,438","$4,558,861.00",...,1.768,48.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Dustin Johnson,36,23,82,20,7,2,"2,493","$4,473,122.00",...,1.767,44.8,37,36,16,323,886,224,3.939,92
4,5,Steve Stricker,54,19,72,19,9,2,"2,028","$4,190,235.50",...,1.746,54.8,,54,5,280,841,157,3.889,259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,,Zander Winston,--,0,0,0,0,0,0,--,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3196,,Connor Howe,--,0,0,0,0,0,0,--,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3197,,Donnie Trosper,--,0,4,0,0,0,0,--,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3198,,Jared C. Kobren,--,0,0,0,0,0,0,--,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
import numpy as np

In [34]:
df_final.replace("--",np.nan,inplace=True)
df_final

,RK,PLAYER,AGE,EVENTS,ROUNDS,CUTS MADE,TOP 10,WINS,CUP POINTS,EARNINGS,...,PUTT AVG.,SAVE PCT.,RK_drop2,AGE_drop2,EAGLES,BIRDIES,PARS,BOGEYS,BIRDIES/RD,HOLES/EAGLE
0,1,Matt Kuchar,42,26,95,23,11,1,"2,728","$4,910,476.50",...,1.739,56.2,121,42,9,377,1111,199,3.968,190
1,2,Jim Furyk,50,21,74,18,7,3,"2,980","$4,809,622.50",...,1.767,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Ernie Els,51,20,70,17,7,2,"1,438","$4,558,861.00",...,1.768,48.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Dustin Johnson,36,23,82,20,7,2,"2,493","$4,473,122.00",...,1.767,44.8,37,36,16,323,886,224,3.939,92
4,5,Steve Stricker,54,19,72,19,9,2,"2,028","$4,190,235.50",...,1.746,54.8,,54,5,280,841,157,3.889,259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,,Zander Winston,NaN,0,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3196,,Connor Howe,NaN,0,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3197,,Donnie Trosper,NaN,0,4,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3198,,Jared C. Kobren,NaN,0,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df_final["AGE"] = df_final["AGE"].fillna(0)

In [36]:
df_final.drop("RK_drop1",axis=1,inplace=True)
df_final.drop("RK_drop2",axis=1,inplace=True)
df_final.drop("AGE_drop1",axis=1,inplace=True)
df_final.drop("AGE_drop2",axis=1,inplace=True)

In [37]:
#deleting 189 rows where earnings is NaN. These appear to be those who played very little 
#likely has very little impact on overall result.
df_final.dropna(subset = ["EARNINGS"], inplace=True)

In [38]:
#1,141 rows fall into this category which is a lot however most are not "primary" players on
#on the PGA tour. Many are European Tour members or those who only played a few events during 
#the year.
df_final.dropna(subset=["YDS/DRIVE","DRIVING ACC.",
                "GREENS IN REG.","PUTT AVG.","SAVE PCT."],inplace=True)   

In [39]:
convert_dict = {"AGE": int,"EVENTS": int, "ROUNDS": int, "CUTS MADE": int, "TOP 10": int, "WINS": int,
                "YEAR":int, "YDS/DRIVE":float, "DRIVING ACC.":float,"GREENS IN REG.":float,"PUTT AVG.":float,
                "SAVE PCT.":float}
df_final = df_final.astype(convert_dict)

In [40]:
#replace 45 "0" ages with mean age of 39
df_final.AGE[df_final.AGE==0] =  39

<ipython-input-40-fbb79196fcd9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.AGE[df_final.AGE==0] =  39


In [41]:
df_final

,RK,PLAYER,AGE,EVENTS,ROUNDS,CUTS MADE,TOP 10,WINS,CUP POINTS,EARNINGS,...,DRVE TOTAL,GREENS IN REG.,PUTT AVG.,SAVE PCT.,EAGLES,BIRDIES,PARS,BOGEYS,BIRDIES/RD,HOLES/EAGLE
0,1,Matt Kuchar,42,26,95,23,11,1,"2,728","$4,910,476.50",...,144,69.4,1.739,56.2,9,377,1111,199,3.968,190
1,2,Jim Furyk,50,21,74,18,7,3,"2,980","$4,809,622.50",...,189,67.1,1.767,48.0,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Ernie Els,51,20,70,17,7,2,"1,438","$4,558,861.00",...,237,67.9,1.768,48.4,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Dustin Johnson,36,23,82,20,7,2,"2,493","$4,473,122.00",...,175,68.0,1.767,44.8,16,323,886,224,3.939,92
4,5,Steve Stricker,54,19,72,19,9,2,"2,028","$4,190,235.50",...,164,68.3,1.746,54.8,5,280,841,157,3.889,259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3092,213,Martin Trainer,29,0,46,2,0,0,31,"$89,482.00",...,45,52.1,1.810,47.9,6,133,465,178,2.891,138
3096,217,Arjun Atwal,48,0,31,5,0,0,32,"$63,252.00",...,172,66.5,1.806,50.0,5,107,350,89,3.452,112
3097,218,Anirban Lahiri,33,0,35,5,0,0,27,"$58,388.00",...,125,68.1,1.767,51.7,2,147,373,89,4.200,315
3098,219,Bo Van Pelt,45,0,43,4,0,0,15,"$50,838.00",...,144,65.1,1.827,25.9,2,128,489,125,2.977,387


In [43]:
df_final.dtypes

RK                 object
PLAYER             object
AGE                 int64
EVENTS              int64
ROUNDS              int64
CUTS MADE           int64
TOP 10              int64
WINS                int64
CUP POINTS         object
EARNINGS           object
YEAR                int64
YDS/DRIVE         float64
DRIVING ACC.      float64
DRVE TOTAL         object
GREENS IN REG.    float64
PUTT AVG.         float64
SAVE PCT.         float64
EAGLES             object
BIRDIES            object
PARS               object
BOGEYS             object
BIRDIES/RD         object
HOLES/EAGLE        object
dtype: object

In [44]:
df_final['EARNINGS'] = df_final['EARNINGS'].replace('[\$,]', '', regex=True).astype(float)

In [45]:
df_final

,RK,PLAYER,AGE,EVENTS,ROUNDS,CUTS MADE,TOP 10,WINS,CUP POINTS,EARNINGS,...,DRVE TOTAL,GREENS IN REG.,PUTT AVG.,SAVE PCT.,EAGLES,BIRDIES,PARS,BOGEYS,BIRDIES/RD,HOLES/EAGLE
0,1,Matt Kuchar,42,26,95,23,11,1,"2,728",4910476.5,...,144,69.4,1.739,56.2,9,377,1111,199,3.968,190
1,2,Jim Furyk,50,21,74,18,7,3,"2,980",4809622.5,...,189,67.1,1.767,48.0,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Ernie Els,51,20,70,17,7,2,"1,438",4558861.0,...,237,67.9,1.768,48.4,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Dustin Johnson,36,23,82,20,7,2,"2,493",4473122.0,...,175,68.0,1.767,44.8,16,323,886,224,3.939,92
4,5,Steve Stricker,54,19,72,19,9,2,"2,028",4190235.5,...,164,68.3,1.746,54.8,5,280,841,157,3.889,259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3092,213,Martin Trainer,29,0,46,2,0,0,31,89482.0,...,45,52.1,1.810,47.9,6,133,465,178,2.891,138
3096,217,Arjun Atwal,48,0,31,5,0,0,32,63252.0,...,172,66.5,1.806,50.0,5,107,350,89,3.452,112
3097,218,Anirban Lahiri,33,0,35,5,0,0,27,58388.0,...,125,68.1,1.767,51.7,2,147,373,89,4.200,315
3098,219,Bo Van Pelt,45,0,43,4,0,0,15,50838.0,...,144,65.1,1.827,25.9,2,128,489,125,2.977,387


In [46]:
duplicate = df_final[df_final.duplicated()]
pd.set_option('display.max_columns', 30)
duplicate

,RK,PLAYER,AGE,EVENTS,ROUNDS,CUTS MADE,TOP 10,WINS,CUP POINTS,EARNINGS,YEAR,YDS/DRIVE,DRIVING ACC.,DRVE TOTAL,GREENS IN REG.,PUTT AVG.,SAVE PCT.,EAGLES,BIRDIES,PARS,BOGEYS,BIRDIES/RD,HOLES/EAGLE


In [47]:
df_final.describe()

,AGE,EVENTS,ROUNDS,CUTS MADE,TOP 10,WINS,EARNINGS,YEAR,YDS/DRIVE,DRIVING ACC.,GREENS IN REG.,PUTT AVG.,SAVE PCT.
count,1870.000000,1870.000000,1870.000000,1870.000000,1870.000000,1870.000000,1.870000e+03,1870.000000,1870.000000,1870.000000,1870.000000,1870.000000,1870.000000
mean,39.436364,23.047059,74.447059,14.574332,2.650267,0.228342,1.477183e+06,2014.918717,291.849251,61.591176,65.277273,1.773649,49.720535
std,7.449273,6.065094,15.225016,4.540948,2.398421,0.572930,1.398625e+06,3.324411,9.089021,5.065117,3.197823,0.026424,6.295905
min,21.000000,0.000000,31.000000,0.000000,0.000000,0.000000,2.487800e+04,2010.000000,266.400000,45.600000,50.200000,1.689000,25.900000
25%,34.000000,21.000000,63.000000,11.000000,1.000000,0.000000,5.637715e+05,2012.000000,285.600000,58.100000,63.400000,1.756000,45.500000
50%,39.000000,24.000000,75.000000,15.000000,2.000000,0.000000,1.030734e+06,2015.000000,291.600000,61.600000,65.600000,1.774000,50.000000
75%,45.000000,27.000000,86.000000,18.000000,4.000000,0.000000,1.892479e+06,2018.000000,297.600000,65.075000,67.500000,1.791000,53.800000
max,61.000000,36.000000,120.000000,27.000000,16.000000,6.000000,1.203046e+07,2020.000000,322.100000,76.900000,74.300000,1.888000,68.700000


In [52]:
df_final.nlargest(10,["EARNINGS"])

,RK,PLAYER,AGE,EVENTS,ROUNDS,CUTS MADE,TOP 10,WINS,CUP POINTS,EARNINGS,YEAR,YDS/DRIVE,DRIVING ACC.,DRVE TOTAL,GREENS IN REG.,PUTT AVG.,SAVE PCT.,EAGLES,BIRDIES,PARS,BOGEYS,BIRDIES/RD,HOLES/EAGLE
1600,1,Jordan Spieth,27,26,92,22,16,6,"6,392",12030465.0,2015,291.8,62.9,78,64.9,1.699,58.1,12,425,1008,184,4.620,138
1920,1,Justin Thomas,27,25,79,18,12,5,"3,000",9921560.0,2017,309.7,55.1,8,67.2,1.694,48.2,14,354,843,187,4.481,102
2560,1,Brooks Koepka,30,21,74,20,9,3,-13,9684006.0,2019,309.0,61.8,10,70.0,1.752,58.4,15,317,820,155,4.284,89
1921,2,Jordan Spieth,27,23,78,19,12,3,"2,340",9433033.0,2017,295.6,60.0,75,70.0,1.711,55.8,8,350,865,160,4.487,176
1601,2,Jason Day,33,21,76,19,12,5,"6,970",9403330.0,2015,313.7,55.9,3,67.1,1.712,61.1,13,358,834,139,4.711,105
1922,3,Dustin Johnson,36,20,70,17,8,4,"1,720",8732193.0,2017,315.0,57.0,2,69.5,1.755,44.3,9,279,802,154,3.986,140
2240,1,Justin Thomas,27,23,82,21,10,3,"1,610",8694821.0,2018,311.9,57.8,0,69.5,1.707,54.2,17,356,825,178,4.341,83
960,1,Tiger Woods,45,16,60,16,8,5,"2,742",8553439.0,2013,293.2,62.5,49,67.6,1.737,60.8,8,244,665,144,4.067,135
2241,2,Dustin Johnson,36,20,74,19,12,3,"2,056",8457352.0,2018,314.0,58.3,0,70.6,1.689,51.2,15,348,729,151,4.703,84
1923,4,Hideki Matsuyama,29,22,76,19,7,3,"1,093",8380570.0,2017,303.3,58.6,26,69.0,1.739,50.9,12,326,834,172,4.289,114


In [54]:
df_final.to_csv('df_final.csv')